In [1]:
from hyperopt import fmin, tpe, hp, partial
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, zero_one_loss

import xgboost as xgb
import pandas as pd

def GetNewDataByPandas():
    wine = pd.read_csv("/home/fonttian/Data/UCI/wine/wine.csv")
    wine['alcohol**2'] = pow(wine["alcohol"], 2)
    wine['volatileAcidity*alcohol'] = wine["alcohol"] * wine['volatile acidity']
    y = np.array(wine.quality)
    X = np.array(wine.drop("quality", axis=1))
    # X = np.array(wine)

    columns = np.array(wine.columns)

    return X, y, columns

In [2]:
# Read wine quality data from file
X, y, wineNames = GetNewDataByPandas()

# split data to [[0.8,0.2],01]
x_train_all, x_predict, y_train_all, y_predict = train_test_split(X, y, test_size=0.10, random_state=100)

x_train, x_test, y_train, y_test = train_test_split(x_train_all, y_train_all, test_size=0.2, random_state=100)

In [3]:
def argsDict_tranform(argsDict, isPrint=False):
    argsDict["max_depth"] = argsDict["max_depth"] + 5
    argsDict['n_estimators'] = argsDict['n_estimators'] + 150
    argsDict["learning_rate"] = argsDict["learning_rate"] * 0.02 + 0.05
    argsDict["subsample"] = argsDict["subsample"] * 0.1 + 0.5
    argsDict["min_child_weight"] = argsDict["min_child_weight"] + 1
    if isPrint:
        print(argsDict)
    else:
        pass

    return argsDict

In [4]:
def xgbRegressor_factory(argsDict):
    argsDict = argsDict_tranform(argsDict)

    xrf = xgb.XGBRegressor(nthread=-1,  # 进程数
                           max_depth=argsDict['max_depth'],  # 最大深度
                           n_estimators=argsDict['n_estimators'],  # 树的数量
                           learning_rate=argsDict['learning_rate'],  # 学习率
                           subsample=argsDict['subsample'],  # 采样数
                           min_child_weight=argsDict['min_child_weight'],  # 终点节点最小样本占比 的 和
                           objective='reg:linear',
                           silent=0,  # 是否显示
                           gamma=0,  # 是否后剪枝
                           colsample_bytree=0.7,  # 样本列采样
                           reg_alpha=0,  # L1 正则化
                           reg_lambda=0,  # L2 正则化
                           scale_pos_weight=0,  # 取值>0时,在数据不平衡时有助于收敛
                           seed=100,  # 随机种子
                           missing=-999 # 填充缺失值
                           )

    return get_tranformer_score(xrf)

In [9]:
def get_tranformer_score(tranformer):
    xrf = tranformer
    
    # without eval_set

    # xlf.fit(x_train_all, y_train_all, eval_metric='rmse', verbose=True)
    # prediction = xlf.predict(x_predict)

    # with eval_set
    xrf.fit(x_train, y_train, eval_metric='rmse', verbose=True, eval_set=[(x_train, y_train), (x_test, y_test)],
            early_stopping_rounds=100)
    prediction = xrf.predict(x_predict, ntree_limit=xrf.best_ntree_limit)
  
    return mean_squared_error(y_predict, prediction)

In [6]:
# 自定义hyperopt的参数空间
space = {"max_depth": hp.randint("max_depth", 15),
         "n_estimators": hp.randint("n_estimators", 300),
         "learning_rate": hp.randint("learning_rate", 6),
         "subsample": hp.randint("subsample", 5),
         "min_child_weight": hp.randint("min_child_weight", 6),
         }

In [12]:
# 开始使用hyperopt进行自动调参
algo = partial(tpe.suggest, n_startup_jobs=1)
best = fmin(xgbRegressor_factory, space, algo=algo, max_evals=20, pass_expr_memo_ctrl=None)

[14:58:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[0]	validation_0-rmse:4.50849	validation_1-rmse:4.62872
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[14:58:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 extra nodes, 0 pruned nodes, max_depth=5
[1]	validation_0-rmse:3.93179	validation_1-rmse:4.04327
[14:58:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[2]	validation_0-rmse:3.43537	validation_1-rmse:3.53932
[14:58:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50 extra nodes, 0 pruned nodes, max_depth=5
[3]	validation_0-rmse:3.00102	validation_1-rmse:3.10603
[14:58:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[

[14:58:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=5
[45]	validation_0-rmse:0.395071	validation_1-rmse:0.603927
[14:58:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=5
[46]	validation_0-rmse:0.392793	validation_1-rmse:0.603092
[14:58:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 0 pruned nodes, max_depth=5
[47]	validation_0-rmse:0.391174	validation_1-rmse:0.602554
[14:58:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 20 extra nodes, 0 pruned nodes, max_depth=5
[48]	validation_0-rmse:0.389924	validation_1-rmse:0.602999
[14:58:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 24 extra nodes, 0 pruned nodes, max_depth=5
[49]	validation_0-rmse:0.388019	validation_1-rmse:0.602421
[14:58:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra no

[14:59:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=5
[91]	validation_0-rmse:0.288173	validation_1-rmse:0.593577
[14:59:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=5
[92]	validation_0-rmse:0.286502	validation_1-rmse:0.59302
[14:59:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 34 extra nodes, 0 pruned nodes, max_depth=5
[93]	validation_0-rmse:0.284724	validation_1-rmse:0.593546
[14:59:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[94]	validation_0-rmse:0.282468	validation_1-rmse:0.592168
[14:59:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=5
[95]	validation_0-rmse:0.280784	validation_1-rmse:0.591053
[14:59:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nod

[14:59:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 38 extra nodes, 0 pruned nodes, max_depth=5
[136]	validation_0-rmse:0.213062	validation_1-rmse:0.585421
[14:59:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 46 extra nodes, 0 pruned nodes, max_depth=5
[137]	validation_0-rmse:0.211589	validation_1-rmse:0.584724
[14:59:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 48 extra nodes, 0 pruned nodes, max_depth=5
[138]	validation_0-rmse:0.209871	validation_1-rmse:0.585344
[14:59:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=5
[139]	validation_0-rmse:0.208959	validation_1-rmse:0.585296
[14:59:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36 extra nodes, 0 pruned nodes, max_depth=5
[140]	validation_0-rmse:0.207699	validation_1-rmse:0.585684
[14:59:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 54 ext

[14:59:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[181]	validation_0-rmse:0.164001	validation_1-rmse:0.585048
[14:59:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 44 extra nodes, 0 pruned nodes, max_depth=5
[182]	validation_0-rmse:0.163187	validation_1-rmse:0.58513
[14:59:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=5
[183]	validation_0-rmse:0.162458	validation_1-rmse:0.585171
[14:59:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52 extra nodes, 0 pruned nodes, max_depth=5
[184]	validation_0-rmse:0.160607	validation_1-rmse:0.585573
[14:59:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=5
[185]	validation_0-rmse:0.160246	validation_1-rmse:0.585479
[14:59:03] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extr

...

[15:02:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 440 extra nodes, 0 pruned nodes, max_depth=19
[23]	validation_0-rmse:1.53496	validation_1-rmse:1.67346
[15:02:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 364 extra nodes, 0 pruned nodes, max_depth=19
[24]	validation_0-rmse:1.46052	validation_1-rmse:1.60169
[15:02:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 448 extra nodes, 0 pruned nodes, max_depth=18
[25]	validation_0-rmse:1.38891	validation_1-rmse:1.53371
[15:02:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 434 extra nodes, 0 pruned nodes, max_depth=19
[26]	validation_0-rmse:1.3216	validation_1-rmse:1.46955
[15:02:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 448 extra nodes, 0 pruned nodes, max_depth=19
[27]	validation_0-rmse:1.25726	validation_1-rmse:1.41053
[15:02:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 434 extra no

[15:02:49] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 424 extra nodes, 0 pruned nodes, max_depth=19
[68]	validation_0-rmse:0.193888	validation_1-rmse:0.596375
[15:02:49] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 382 extra nodes, 0 pruned nodes, max_depth=19
[69]	validation_0-rmse:0.186886	validation_1-rmse:0.594404
[15:02:49] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 376 extra nodes, 0 pruned nodes, max_depth=19
[70]	validation_0-rmse:0.180167	validation_1-rmse:0.592175
[15:02:49] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 320 extra nodes, 0 pruned nodes, max_depth=19
[71]	validation_0-rmse:0.173944	validation_1-rmse:0.590593
[15:02:49] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 352 extra nodes, 0 pruned nodes, max_depth=19
[72]	validation_0-rmse:0.167966	validation_1-rmse:0.588716
[15:02:49] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 3

[15:02:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 292 extra nodes, 0 pruned nodes, max_depth=19
[113]	validation_0-rmse:0.06175	validation_1-rmse:0.56918
[15:02:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 270 extra nodes, 0 pruned nodes, max_depth=19
[114]	validation_0-rmse:0.060869	validation_1-rmse:0.568976
[15:02:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 284 extra nodes, 0 pruned nodes, max_depth=19
[115]	validation_0-rmse:0.059948	validation_1-rmse:0.569356
[15:02:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 318 extra nodes, 0 pruned nodes, max_depth=19
[116]	validation_0-rmse:0.058877	validation_1-rmse:0.56944
[15:02:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 366 extra nodes, 0 pruned nodes, max_depth=19
[117]	validation_0-rmse:0.057857	validation_1-rmse:0.56954
[15:02:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 

[15:02:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 326 extra nodes, 0 pruned nodes, max_depth=19
[158]	validation_0-rmse:0.032622	validation_1-rmse:0.570803
[15:02:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 288 extra nodes, 0 pruned nodes, max_depth=19
[159]	validation_0-rmse:0.032261	validation_1-rmse:0.570902
[15:02:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 348 extra nodes, 0 pruned nodes, max_depth=19
[160]	validation_0-rmse:0.031871	validation_1-rmse:0.571013
[15:02:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 296 extra nodes, 0 pruned nodes, max_depth=19
[161]	validation_0-rmse:0.031536	validation_1-rmse:0.571038
[15:02:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 404 extra nodes, 0 pruned nodes, max_depth=19
[162]	validation_0-rmse:0.031089	validation_1-rmse:0.571046
[15:02:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roo

[15:02:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 262 extra nodes, 0 pruned nodes, max_depth=19
[203]	validation_0-rmse:0.018957	validation_1-rmse:0.571743
[15:02:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 296 extra nodes, 0 pruned nodes, max_depth=19
[204]	validation_0-rmse:0.018742	validation_1-rmse:0.571766
[15:02:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 234 extra nodes, 0 pruned nodes, max_depth=19
[205]	validation_0-rmse:0.018579	validation_1-rmse:0.571818
[15:02:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 156 extra nodes, 0 pruned nodes, max_depth=19
[206]	validation_0-rmse:0.018353	validation_1-rmse:0.571838
[15:02:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 342 extra nodes, 0 pruned nodes, max_depth=19
[207]	validation_0-rmse:0.01808	validation_1-rmse:0.571893
[15:02:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 root

In [13]:
# 展示结果
RMSE = xgbRegressor_factory(best)
# print('mse of the best xgboost:', RMSE)
print('best :', best)
print('best param after transform :')
argsDict_tranform(best,isPrint=True)
print('rmse of the best xgboost:', np.sqrt(RMSE))

[15:03:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 338 extra nodes, 0 pruned nodes, max_depth=15
[0]	validation_0-rmse:4.91526	validation_1-rmse:5.04788
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 100 rounds.
[15:03:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 444 extra nodes, 0 pruned nodes, max_depth=17
[1]	validation_0-rmse:4.67129	validation_1-rmse:4.79958
[15:03:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 438 extra nodes, 0 pruned nodes, max_depth=17
[2]	validation_0-rmse:4.43921	validation_1-rmse:4.56441
[15:03:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 442 extra nodes, 0 pruned nodes, max_depth=17
[3]	validation_0-rmse:4.21875	validation_1-rmse:4.34163
[15:03:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 444 extra nodes, 0 pruned nodes, max_

[15:03:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 370 extra nodes, 0 pruned nodes, max_depth=17
[45]	validation_0-rmse:0.523922	validation_1-rmse:0.773804
[15:03:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 410 extra nodes, 0 pruned nodes, max_depth=17
[46]	validation_0-rmse:0.500048	validation_1-rmse:0.758198
[15:03:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 420 extra nodes, 0 pruned nodes, max_depth=17
[47]	validation_0-rmse:0.477485	validation_1-rmse:0.743891
[15:03:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 442 extra nodes, 0 pruned nodes, max_depth=17
[48]	validation_0-rmse:0.455605	validation_1-rmse:0.728388
[15:03:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 388 extra nodes, 0 pruned nodes, max_depth=17
[49]	validation_0-rmse:0.43523	validation_1-rmse:0.715284
[15:03:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 36

[89]	validation_0-rmse:0.102355	validation_1-rmse:0.574825
[15:03:37] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 202 extra nodes, 0 pruned nodes, max_depth=17
[90]	validation_0-rmse:0.100101	validation_1-rmse:0.574581
[15:03:37] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 258 extra nodes, 0 pruned nodes, max_depth=17
[91]	validation_0-rmse:0.097889	validation_1-rmse:0.573989
[15:03:37] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 454 extra nodes, 0 pruned nodes, max_depth=17
[92]	validation_0-rmse:0.095292	validation_1-rmse:0.573857
[15:03:37] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 308 extra nodes, 0 pruned nodes, max_depth=17
[93]	validation_0-rmse:0.093121	validation_1-rmse:0.573574
[15:03:37] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 382 extra nodes, 0 pruned nodes, max_depth=17
[94]	validation_0-rmse:0.09073	validation_1-rmse:0.573105
[15:03:37] /workspace/

[15:03:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 216 extra nodes, 0 pruned nodes, max_depth=17
[135]	validation_0-rmse:0.046419	validation_1-rmse:0.570232
[15:03:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 340 extra nodes, 0 pruned nodes, max_depth=17
[136]	validation_0-rmse:0.045776	validation_1-rmse:0.570297
[15:03:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 348 extra nodes, 0 pruned nodes, max_depth=17
[137]	validation_0-rmse:0.045075	validation_1-rmse:0.570324
[15:03:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 400 extra nodes, 0 pruned nodes, max_depth=17
[138]	validation_0-rmse:0.044239	validation_1-rmse:0.570533
[15:03:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 276 extra nodes, 0 pruned nodes, max_depth=17
[139]	validation_0-rmse:0.043542	validation_1-rmse:0.570644
[15:03:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roo

[15:03:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 184 extra nodes, 0 pruned nodes, max_depth=17
[180]	validation_0-rmse:0.025446	validation_1-rmse:0.571215
[15:03:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 282 extra nodes, 0 pruned nodes, max_depth=17
[181]	validation_0-rmse:0.025106	validation_1-rmse:0.571267
[15:03:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 222 extra nodes, 0 pruned nodes, max_depth=17
[182]	validation_0-rmse:0.024871	validation_1-rmse:0.571363
[15:03:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 0 pruned nodes, max_depth=17
[183]	validation_0-rmse:0.024751	validation_1-rmse:0.571384
[15:03:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 290 extra nodes, 0 pruned nodes, max_depth=17
[184]	validation_0-rmse:0.02442	validation_1-rmse:0.571357
[15:03:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots

[15:03:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 204 extra nodes, 0 pruned nodes, max_depth=17
[225]	validation_0-rmse:0.015703	validation_1-rmse:0.572049
[15:03:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 196 extra nodes, 0 pruned nodes, max_depth=17
[226]	validation_0-rmse:0.015567	validation_1-rmse:0.572032
best : {'learning_rate': 0.05, 'max_depth': 17, 'min_child_weight': 3, 'n_estimators': 227, 'subsample': 0.7}
best param after transform :
{'learning_rate': 0.051000000000000004, 'max_depth': 22, 'min_child_weight': 4, 'n_estimators': 377, 'subsample': 0.57}
rmse of the best xgboost: 0.5389822960167566


In [16]:
"""
best : {'learning_rate': 0.05, 'max_depth': 17, 'min_child_weight': 3, 'n_estimators': 227, 'subsample': 0.7}
best param after transform :
{'learning_rate': 0.051000000000000004, 'max_depth': 22, 'min_child_weight': 4, 'n_estimators': 377, 'subsample': 0.57}
rmse of the best xgboost: 0.538982296016756
"""

"\nbest : {'colsample_bytree': 0.7, 'gamma': 0.0, 'learning_rate': 0.15000000000000002, 'max_depth': 16, 'min_child_weight': 1, 'n_estimators': 440, 'subsample': 0.8}\nbest param after transform :\n{'colsample_bytree': 0.57, 'gamma': 0.0, 'learning_rate': 0.053000000000000005, 'max_depth': 21, 'min_child_weight': 2, 'n_estimators': 590, 'subsample': 0.5800000000000001}\nrmse of the best xgboost: 0.5602862515362897\n"